In [ ]:
# default_exp initalise_repository

# initalise_repository
> This script sets up the inital repository.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *
from nbdev import *

In [ ]:
#export

#C--------------------------------------------------------------------------
#C  Program : initalise_repository.py
#C 
#C  Setsup inital repsoitory
#C 
#C  Copyright: 3ideas.co.uk  
#C
#C  See https://github.com/3ideas/config_tracker
#C--------------------------------------------------------------------------

from distutils.dir_util import copy_tree
import argparse
from os import listdir
from os.path import isfile, join
import os
import time
import datetime
import sys
from subprocess import call


def run_subprocess(command):
    """
    Run the subprocess
    """
    try:

        retcode = call(command, shell=True)
        print("RAN: %s       Returned: %s" % (command,retcode))
        if retcode < 0:
            print("CMD: %s, was terminated by signal: " % command, -
                  retcode, file=sys.stderr)
        elif retcode > 0:
            print("CMD: %s, returned: " % command , retcode, file=sys.stderr)
    except OSError as e:
        print("Execution failed:", e, file=sys.stderr)
    return retcode


def create_base_directories(directory):

    if not os.path.isdir('files'):
        os.mkdir('files')
    if not os.path.isdir('database'):
        os.mkdir('database')


def directory_is_empty(directory):
    """
    Check if the directory is empty
    """
    if len(os.listdir(directory)) == 0:
        return True
    return False


def parse_directory_date_to_datetime(directory):
    """
    Parse the directory name to a timestamp
    """
    if len(directory) != 8:
        return None

    try:
        year = int(directory[0:4])
        month = int(directory[4:6])
        date = int(directory[6:8])
        dt = datetime.datetime(year=year, month=month, day=date)
    except:
        return None
    return dt


def datetime_to_timestamp(dt):
    return time.mktime(dt.timetuple())


def delete_files_in_repository_directory(directory):
    """
    Delete all files in the repository directory
    """
    print(f'Deleting from directory: {directory}')
    for file in os.scandir(directory):
        # TODO: check if the file is a directory, and remove the directory ...
        # print(f'deleting: {file.path}')
        os.unlink(file.path)


def copy_directory_to_directory(source, destination):
    """
    Copy the directory to the destination, renaming all txt files to csv
    """
    copy_tree(source, destination)


def rename_all_files_in_directory_from_txt_to_csv(directory):
    """
    Rename all files in the directory .txt with .csv
    """
    for file in os.scandir(directory):
        if file.is_file() and file.name.endswith('.txt'):
            f_new = file.path.replace('.txt', '.csv')
            os.rename(file.path, f_new)

    # rename all the txt files to .csv
    # for f in os.listdir(destdirectoryination):
    #    if f.endswith('.txt'):
    #        f_new = f.replace('.txt', '.csv')
    #        os.rename(join(directory, f), join(directory, f_new))


def add_all_new_files_to_respository(directory):
    """
    Add the files to the repository
    """
    result = run_subprocess('git add .')
    return result


def commit_repository(directory, comment):
    """
    Commit the repository
    """
    result = run_subprocess('git commit -m "%s"' %comment)
    return result


def branch_exists(directory, branch):
    """
    Check if the branch exists
    """
    result = run_subprocess(
        'git rev-parse --verify %s --quiet 2>/dev/null' %branch )
    if result != 0:
        return False
    return True

    return run_subprocess('git rev-parse --verify %s --quiet 2>/dev/null'%branch)


def create_branch(directory, branch):
    """
    Create the branch
    """
    if switch_to_branch(repository, 'master') != 0:
        print('create_branch: Failed to switch to master branch')
        return 1

    result = run_subprocess(f'git branch {branch}')
    if result != 0:
        print('create_branch: Failed to create branch:%s'%branch)
        return result
    if switch_to_branch(repository, branch) != 0:
        print(
            'create_branch: Failed to switch to branch after creating it: %s'%branch)
        return 1
    result = run_subprocess(
        "echo 'This is the branch for: %s, we should only see this file in this branch!' >%s.md"%(branch,branch))
    create_base_directories(directory)
    add_all_new_files_to_respository(directory)
    commit_repository(
        directory, "create_branch: Initial commit, of branch: %s"%branch)
    return result


def switch_to_branch(directory, branch):
    result = run_subprocess(f'git checkout {branch}')
    return result


def repository_exists(directory):
    """
    Check if the repository exists
    """
    # check if .git and is a directory

    if os.path.isdir('.git'):
        return True
    return False


def create_repository(directory):
    """
    Create the repository
    """
    # TODO: create directory if it does not exist

    # mkdir -p $BASE_DIR
    # cd $BASE_DIR
    # git init -b master
    # echo "Adding initial marker README.md file in repository"
    # echo 'This is the repository for all configuration. There should be a git branch for every server recorded here, the local server should be the active branch' >README.md
    # git add .
    # git commit -m 'Initial commit, of readme'

    result = run_subprocess('git init')
    if result != 0:
        print('Failed to create repository')
        exit(result)
    result = run_subprocess(
        "echo 'This is the repository for all configuration. There should be a git branch for every server recorded here, the local server should be the active branch' >README.md")
    if result != 0:
        print('Failed to create repository')

    add_all_new_files_to_respository(directory)
    commit_repository(directory, 'Initial commit, of readme')
    return result


def add_directory_to_repository(repository, server, dt, tag, source, destination):
    """
    Copy the directory to the repository in the current branch and commit it.
    """

    date_str = dt.strftime("%Y-%m-%d")
    result = run_subprocess(
        "echo 'Last updated for: %s at %s  (%s)' >last_updated.md" %(server,date_str,tag))
    if result != 0:
        print('Failed to create repository')

    delete_files_in_repository_directory(destination)

    # TODO split into seperate function to copy directories and files
    # Copy files from one directory to another

    copy_directory_to_directory(source, destination)
    rename_all_files_in_directory_from_txt_to_csv(destination)

    result = add_all_new_files_to_respository(repository)
    if result != 0:
        print("Failed to add files to the repository")
        exit(1)
    result = commit_repository(repository, f'{tag}')
    if result != 0:
        print("Failed to commit the repository")
        exit(1)


def copy_into_repository(repository, exportsdir, server, force):
    """
    Copy the files into the repository
    """

    path = exportsdir + '/' + server

    database_repository = repository + '/database/'

    if switch_to_branch(repository, server) < 0:
        exit(1)

    if os.path.isdir(database_repository):
        if not force and not directory_is_empty(database_repository):
            print("""The database repository is not empty. Please empty it first. 
            This is because this directory is cleared after  every load and to ensure we don't delete anything unintended, 
            it should be cleared""")
            exit(1)
    else:
        os.mkdir('database')

    dirs = [f for f in listdir(path)]
    dirs.sort()
    for d in dirs:
        dt = parse_directory_date_to_datetime(d)
        if dt == None:
            print("ignoring:%s"%d)
            continue

        print('Timestamp:%s'%dt)

        # watch the rename for when we do the files!!
        add_directory_to_repository(repository, server, dt, d,
                                    path + '/' + d, database_repository)





In [ ]:
#export
try: from nbdev.imports import IN_NOTEBOOK
except: IN_NOTEBOOK=False
    
if __name__ == "__main__" and not IN_NOTEBOOK:
    ap = argparse.ArgumentParser()
    ap.add_argument("-s", "--server", required=True, help="name of the server")
    ap.add_argument("-e", "--exportsdir", required=False,
                help="directory of the current config to be copied into repository")
    ap.add_argument("-r", "--repository", required=True,
                help="directory of the repository")
    ap.add_argument("-f", "--force", required=False, default=False, action='store_true',
                help="force the operation")
    ap.add_argument("-i", "--initalise", required=False, default=False, action='store_true',
                help="Initialiase the repository, creates master branch, ")

    ap.add_argument("-b", "--branch", required=False, default=False, action='store_true',
                help="creates the server branch")


    args = vars(ap.parse_args())

    server = args["server"]
    exportsdir = args["exportsdir"]
    repository = args["repository"]
    force = args["force"]
    initalise_repository = args["initalise"]
    create_branch_flg = args["branch"]

    os.chdir(repository)  # TODO if this does not exist create it?

    if initalise_repository:
        if repository_exists(repository):
            print('Repository %s already exists'%repository)
            sys.exit(1)
        print('Creating repository: %s'%repository)
        create_repository(repository)
        print('Repository: %s created'%repository)
        print('Note any other cmd line options ignored!')
        exit(0)


    if not repository_exists(repository):
        print('Repository %s does not exist'%repository)
        exit(1)


    # Create a branch for the server's data that is being imported
    if exportsdir and branch_exists(repository, server) and not force:
        print("branch for server already exists, probably don't want to do this. exiting. To overide this use --force")
        exit(1)

    if create_branch_flg:
        # switch to master first
        if switch_to_branch(repository, 'master') != 0:
            print('Failed to switch to master branch')
            exit(1)
        if not branch_exists(repository, server):
            if create_branch(repository, server) != 0:
                print('Failed to create branch: %s'%server)
                exit(1)
            print('Branch %s created'%server)
        else:
            print('Branch: %s already exists, not created'%server)

    if switch_to_branch(repository, server) != 0:
        print('Failed to switch to branch: %s'%server)
        print('Branch needs to be created by specifying the -b option to create the branch')
        exit(1)

    if exportsdir:
        copy_into_repository(repository, exportsdir, server, force)
        


In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_csv_header_restore.ipynb.
Converted 01_convert_to_csv.ipynb.
Converted 02_initalise_repository.ipynb.
Converted 03_convert_files.ipynb.
Converted 04_generate_report1.py.ipynb.
Converted index.ipynb.


In [ ]:
!cat bash_python_header.txt > scripts/initalise_repository.py
!cat  config_tracker/initalise_repository.py >>scripts/initalise_repository.py
!chmod +x scripts/initalise_repository.py